In [1]:
import os
import json
import mne
import scipy

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

from mne.preprocessing import ICA
from mne.time_frequency import psd_array_welch
from scipy.signal import hilbert, coherence

try:
    import PyQt5.QtCore
    %matplotlib qt
except ImportError:
    %matplotlib inline

mne.set_log_level('WARNING')

In [2]:
import my_functions as my_fun
import my_visualization_functions as my_vis_fun

In [ ]:
# Define subjects to process
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 31)]  
data_path = "eeg-motor-movementimagery-dataset-1.0.0/files/"

In [ ]:
subjects.remove("S038")

In [ ]:
# Process EEG data
eeg_data = my_fun.process_eeg(subjects=subjects, data_path=data_path, mode="automatic", apply_ica=True)

## Visualization for one subject

In [ ]:
subject = "S001" 
condition = "imagined_right_hand"  # real_left_hand or imagined_left_hand
epochs = eeg_data[subject][condition][0]

In [ ]:
my_vis_fun.plot_raw_eeg(eeg_data[subject][condition][0], subject, condition)

In [ ]:
my_vis_fun.plot_erd_ers(epochs, subject, condition, motor_channels=["C3", "C4"])

## PLV and Coherence Pipeline for one subject

In [ ]:
real_epochs = mne.concatenate_epochs(eeg_data[subject]['real_right_hand'])
imagined_epochs = mne.concatenate_epochs(eeg_data[subject]['imagined_right_hand'])
rest_epochs = mne.concatenate_epochs(eeg_data[subject]['rest'])

In [ ]:
plv_real = my_fun.compute_plv_matrix(real_epochs)
plv_imagined = my_fun.compute_plv_matrix(imagined_epochs)
plv_rest = my_fun.compute_plv_matrix(rest_epochs)

In [ ]:
my_vis_fun.plot_plv_matrix(plv_real, real_epochs.ch_names, title="PLV - Real Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined, imagined_epochs.ch_names, title="PLV - Imagined Right Hand")
my_vis_fun.plot_plv_matrix(plv_rest, rest_epochs.ch_names, title="PLV - Rest")

In [ ]:
my_vis_fun.plot_plv_difference(plv_real, plv_imagined, real_epochs.ch_names)
my_vis_fun.plot_plv_difference(plv_real, plv_rest, real_epochs.ch_names, title="PLV Difference (Real - Rest)")

In [ ]:
my_vis_fun.plot_motor_plv_difference(plv_real, plv_imagined, real_epochs.ch_names, title="PLV (Motor Cortex) - Real Right Hand")

## PLV and Coherence Pipeline for multiple subjects

In [ ]:
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 3)]  

plv_results = {"real": [], "imagined": [], "rest": []}

eeg_data_all = {}  # <-- Store all subjects

for subject in subjects:
    print(f"\n🚀 Processing {subject}...")
    
    eeg_data = my_fun.process_eeg([subject], data_path="eeg-motor-movementimagery-dataset-1.0.0/files/", mode="automatic", apply_ica=False)

    # Merge subject's data into master dict
    eeg_data_all.update(eeg_data)

    # Compute PLV matrix
    real_epochs = mne.concatenate_epochs(eeg_data[subject]["real_right_hand"])
    imagined_epochs = mne.concatenate_epochs(eeg_data[subject]["imagined_right_hand"])
    rest_epochs = mne.concatenate_epochs(eeg_data[subject]["rest"])

    plv_real = my_fun.compute_plv_matrix(real_epochs)
    plv_imagined = my_fun.compute_plv_matrix(imagined_epochs)
    plv_rest = my_fun.compute_plv_matrix(rest_epochs)

    plv_results["real"].append(plv_real)
    plv_results["imagined"].append(plv_imagined)
    plv_results["rest"].append(plv_rest)


In [ ]:
plv_real_group = np.mean(plv_results["real"], axis=0)
plv_imagined_group = np.mean(plv_results["imagined"], axis=0)
plv_rest_group = np.mean(plv_results["rest"], axis=0)

In [ ]:
my_vis_fun.plot_plv_matrix(plv_real_group, imagined_epochs.ch_names, title="Group Average PLV - Real Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined_group, imagined_epochs.ch_names, title="Group Average PLV - Imagined Right Hand")
my_vis_fun.plot_plv_matrix(plv_imagined_group, imagined_epochs.ch_names, title="Group Average PLV - Rest")

my_vis_fun.plot_plv_difference(plv_real_group, plv_imagined_group, real_epochs.ch_names)
my_vis_fun.plot_plv_difference(plv_real_group, plv_rest_group, real_epochs.ch_names, title="PLV Difference (Real - Rest)")

#my_vis_fun.plot_motor_plv_difference(plv_real_group, plv_imagined_group, real_epochs.ch_names)

In [ ]:
# Define subjects and channel pairs
#subjects = ["S001", "S002", "S005"]
conditions = {"real": "real_right_hand", "imagined": "imagined_right_hand", "rest": "rest"} 

channel_pairs = [
    ("C3", "C4"),  # Primary motor cortex
    ("C1", "C2"),  # Central midline
    ("C5", "C6"),  # Central lateral
    ("FC3", "FC4"),  # Frontal-motor
    ("Cz", "CPz"),  # Midline execution
    ("CP3", "CP4"),  # Parietal-motor
    ("Fz", "Cz"),  # Frontal-central connection
    ("Fp1", "Fp2"),  # Frontal
    ("AF3", "AF4"),  # Anterior frontal
    ("O1", "O2"),  # Occipital 
    ("PO7", "PO8"),  # Parietal
    ("Fp1", "PO8"),   # Prefrontal
    ("T9", "T10"),  # Temporal
]

# Analyze
df_plv_coh = my_fun.analyze_pairwise_plv_coherence(subjects, eeg_data_all, conditions, channel_pairs)

# Plot!
my_vis_fun.plot_plv_coherence(df_plv_coh, metric="PLV Mean")
my_vis_fun.plot_plv_coherence(df_plv_coh, metric="Coherence Mean")

In [ ]:
df_stats = my_fun.paired_ttest_plv(df_plv_coh, metric="PLV Mean")
my_fun.report_paired_ttests(df_stats)

In [ ]:
my_fun.plot_normalized_plv_coherence(df_plv_coh)

In [ ]:
my_fun.plot_ttest_summary(df_stats)

# iPLV and wPLI metrics for multiple subjects

Only uses the imaginary part of the phase difference, which is theoretically insensitive to zero-lag connections
Reduces impact of volume conduction and reference electrode effects
More specific for true neural connectivity than regular PLV
Values range from 0 (no connectivity) to 1 (perfect phase synchronization)


Weights phase differences by the magnitude of the imaginary component
Less sensitive to noise than other phase metrics
Highly resistant to volume conduction effects
Robust measure of true neural connectivity
Values range from 0 (no connectivity) to 1 (perfect phase synchronization)

PLV can produce spurious connectivity due to volume conduction
iPLV and wPLI are more conservative but more specific measures
You should expect overall lower values with iPLV and wPLI compared to PLV
The pattern of significant connections might change substantially as non-neural connections are filtered out

In [ ]:
# Define subjects and initialize storage
subjects = [f"S{str(i).zfill(3)}" for i in range(1, 31)]
connectivity_results = {
    "real": {"iplv": [], "wpli": []},
    "imagined": {"iplv": [], "wpli": []},
    "rest": {"iplv": [], "wpli": []}
}
eeg_data_all = {}  # Store all subjects data

# Process each subject
for subject in subjects:
    print(f"\n🚀 Processing {subject}...")
    
    # Process EEG data
    eeg_data = my_fun.process_eeg([subject], data_path="eeg-motor-movementimagery-dataset-1.0.0/files/", 
                          mode="automatic", apply_ica=False)
    
    # Merge subject's data into master dict
    eeg_data_all.update(eeg_data)
    
    # Only continue if we have data for this subject
    if subject not in eeg_data or not all(cond in eeg_data[subject] for cond in 
                                         ["real_right_hand", "imagined_right_hand", "rest"]):
        print(f"⚠️ Missing condition data for {subject}, skipping connectivity analysis")
        continue
    
    # Concatenate epochs for each condition
    real_epochs = mne.concatenate_epochs(eeg_data[subject]["real_right_hand"])
    imagined_epochs = mne.concatenate_epochs(eeg_data[subject]["imagined_right_hand"])
    rest_epochs = mne.concatenate_epochs(eeg_data[subject]["rest"])
    
    # Compute iPLV matrices
    iplv_real = my_fun.compute_iplv_matrix(real_epochs)
    iplv_imagined = my_fun.compute_iplv_matrix(imagined_epochs)
    iplv_rest = my_fun.compute_iplv_matrix(rest_epochs)
    
    # Compute wPLI matrices
    wpli_real = my_fun.compute_wpli_matrix(real_epochs)
    wpli_imagined = my_fun.compute_wpli_matrix(imagined_epochs)
    wpli_rest = my_fun.compute_wpli_matrix(rest_epochs)

    # Store results
    connectivity_results["real"]["iplv"].append(iplv_real)
    connectivity_results["real"]["wpli"].append(wpli_real)
    
    connectivity_results["imagined"]["iplv"].append(iplv_imagined)
    connectivity_results["imagined"]["wpli"].append(wpli_imagined)
    
    connectivity_results["rest"]["iplv"].append(iplv_rest)
    connectivity_results["rest"]["wpli"].append(wpli_rest)
    
    print(f"✅ Computed iPLV and wPLI for {subject}")


# Compute group averages
iplv_real_group = np.mean(connectivity_results["real"]["iplv"], axis=0)
iplv_imagined_group = np.mean(connectivity_results["imagined"]["iplv"], axis=0)
iplv_rest_group = np.mean(connectivity_results["rest"]["iplv"], axis=0)

wpli_real_group = np.mean(connectivity_results["real"]["wpli"], axis=0)
wpli_imagined_group = np.mean(connectivity_results["imagined"]["wpli"], axis=0)
wpli_rest_group = np.mean(connectivity_results["rest"]["wpli"], axis=0)

# Plot group connectivity matrices
my_fun.plot_connectivity_matrix(iplv_real_group, imagined_epochs.ch_names, 
                        title="Group Average iPLV - Real Right Hand")
my_fun.plot_connectivity_matrix(iplv_imagined_group, imagined_epochs.ch_names, 
                        title="Group Average iPLV - Imagined Right Hand")
my_fun.plot_connectivity_matrix(iplv_rest_group, imagined_epochs.ch_names, 
                        title="Group Average iPLV - Rest")

my_fun.plot_connectivity_matrix(wpli_real_group, imagined_epochs.ch_names, 
                        title="Group Average wPLI - Real Right Hand")
my_fun.plot_connectivity_matrix(wpli_imagined_group, imagined_epochs.ch_names, 
                        title="Group Average wPLI - Imagined Right Hand")
my_fun.plot_connectivity_matrix(wpli_rest_group, imagined_epochs.ch_names, 
                        title="Group Average wPLI - Rest")

# Plot connectivity differences
my_fun.plot_connectivity_difference(iplv_real_group, iplv_imagined_group, imagined_epochs.ch_names,
                           title="iPLV Difference (Real - Imagined)")
my_fun.plot_connectivity_difference(iplv_real_group, iplv_rest_group, imagined_epochs.ch_names,
                           title="iPLV Difference (Real - Rest)")

my_fun.plot_connectivity_difference(wpli_real_group, wpli_imagined_group, imagined_epochs.ch_names,
                           title="wPLI Difference (Real - Imagined)")
my_fun.plot_connectivity_difference(wpli_real_group, wpli_rest_group, imagined_epochs.ch_names,
                           title="wPLI Difference (Real - Rest)")

# Define channel pairs and conditions for pairwise analysis
channel_pairs = [
    ("C3", "C4"),   # Primary motor cortex
    ("C1", "C2"),   # Central midline
    ("C5", "C6"),   # Central lateral
    ("FC3", "FC4"), # Frontal-motor
    ("Cz", "CPz"),  # Midline execution
    ("CP3", "CP4"), # Parietal-motor
    ("Fz", "Cz"),   # Frontal-central connection
    ("Fp1", "Fp2"), # Frontal
    ("AF3", "AF4"), # Anterior frontal
    ("O1", "O2"),   # Occipital 
    ("PO7", "PO8"), # Parietal
    ("Fp1", "PO8"),  # Prefrontal
    ("T9", "T10"),  # Temporal
]

conditions = {"real": "real_right_hand", "imagined": "imagined_right_hand", "rest": "rest"}

# Analyze pairwise connectivity
df_connectivity = my_fun.analyze_pairwise_connectivity(
    subjects, 
    eeg_data_all, 
    conditions, 
    channel_pairs, 
    metrics=["iPLV", "wPLI"]
)

# Plot the connectivity metrics
my_fun.plot_iplv_wpli_comparison(df_connectivity, metrics=["iPLV Mean", "wPLI"])

# Plot normalized connectivity (relative to rest)
my_fun.plot_normalized_connectivity(df_connectivity, metrics=["iPLV Mean", "wPLI"])

# Perform statistical testing
df_stats_iplv = my_fun.paired_ttest_connectivity(df_connectivity, metric="iPLV Mean")
df_stats_wpli = my_fun.paired_ttest_connectivity(df_connectivity, metric="wPLI")

# Report statistical results
my_fun.report_paired_ttests(df_stats_iplv)
my_fun.report_paired_ttests(df_stats_wpli)

# Plot t-test summary for iPLV
my_fun.plot_ttest_summary(df_stats_iplv)

# Plot t-test summary for wPLI
my_fun.plot_ttest_summary(df_stats_wpli)


🚀 Processing S001...

🔄 Processing S001...
✅ Extracted 7 epochs for S001 - R03 (real_right_hand)
✅ Extracted 14 rest epochs for S001 - R03
✅ Extracted 7 epochs for S001 - R07 (real_right_hand)
✅ Extracted 14 rest epochs for S001 - R07
✅ Extracted 8 epochs for S001 - R11 (real_right_hand)
✅ Extracted 14 rest epochs for S001 - R11
✅ Extracted 7 epochs for S001 - R04 (imagined_right_hand)
✅ Extracted 14 rest epochs for S001 - R04
✅ Extracted 7 epochs for S001 - R08 (imagined_right_hand)
✅ Extracted 14 rest epochs for S001 - R08
✅ Extracted 8 epochs for S001 - R12 (imagined_right_hand)
✅ Extracted 14 rest epochs for S001 - R12

✅ Processing complete.
✅ Computed iPLV and wPLI for S001

🚀 Processing S002...

🔄 Processing S002...
✅ Extracted 7 epochs for S002 - R03 (real_right_hand)
✅ Extracted 14 rest epochs for S002 - R03
✅ Extracted 8 epochs for S002 - R07 (real_right_hand)
✅ Extracted 14 rest epochs for S002 - R07
✅ Extracted 7 epochs for S002 - R11 (real_right_hand)
✅ Extracted 14 rest 

c:\Users\flavi\Desktop\GABS Project\EEG-project\my_functions.py:805: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  annot=pval_matrix.applymap(lambda x: f"{x:.4f}"),
c:\Users\flavi\Desktop\GABS Project\EEG-project\my_functions.py:805: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  annot=pval_matrix.applymap(lambda x: f"{x:.4f}"),


# Time Frequency Representation Analysis

In [ ]:
# For each condition (real, imagined, rest)
freqs = np.arange(6, 30, 1)  # 5-40 Hz range
n_cycles = freqs / 2

power_real = my_fun.tfr_morlet(real_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)
power_imagined = my_fun.tfr_morlet(imagined_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)
power_rest = my_fun.tfr_morlet(rest_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False)

In [ ]:
# Plot time-frequency plots for key channels (C3, C4)
power_real.plot_joint(title='Real Movement', picks=['Cz', 'CPz'])
power_imagined.plot_joint(title='Imagined Movement', picks=['Cz', 'CPz'])
power_rest.plot_joint(title='Rest', picks=['Cz', 'CPz'])

# Classification / Decoding Part

In [ ]:
# 1. Run classification on both condition pairs
group_results, individual_results = my_fun.classify_condition_pairs(eeg_data, subjects)

# 2. Visualize the comparison
summary = my_vis_fun.visualize_comparison(group_results, individual_results)

# Frequency Bands and Time Window Analysis

In [ ]:
# Run the frequency band analysis
band_results, band_names = my_fun.analyze_frequency_bands(eeg_data, subjects)

In [ ]:
# Run the time window analysis
window_results, window_names = my_fun.analyze_time_windows(eeg_data, subjects)

# RDM Analysis